# Step 1c - Calculate Transport Costs

This notebook calculates **economic** transport costs based on RED / HDM4 VOCs ($ / kg) over road lengths

In [1]:
import geopandas as gpd
import pandas as pd
import os, sys

# sys.path.append("../../../GOSTnets/GOSTnets")

import GOSTnets as gn
from GOSTnets.load_osm import *
import importlib
import networkx as nx
import osmnx as ox
from shapely import ops as ops
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

### Setup

Define filepaths

In [4]:
input_pth = r'..\..\..\GEO'
lcl_input_pth = r'inputs'
interm_pth = r'intermediate'
fin_pth = r'final'

osm_fil = r'bangladesh_210329_osm.pbf'

f = os.path.join(input_pth,'OSM',osm_fil) 

Define parameters

In [5]:
simplif_meters = 25 # a bit of a manual process, higher number = simpler but more chance for errors
target_crs = 3106 # using Gulshan 303 / Bangladesh Traverse Mercator as it's a national metric projection with an established EPSG

In [6]:
# Production date for outputs being used

# prod_date = datetime.today().strftime('%y%m%d')
# prod_date = '210312'
prod_date = '210329'


### Load in data and transform

In [26]:
largest_G = nx.read_gpickle(os.path.join(interm_pth, 'largest_G_{}m_{}.pickle').format(simplif_meters,prod_date))

In [27]:
## define a new transport cost function
def convert_network_to_transport_cost(G, distance_tag, graph_type = 'drive', road_col = 'highway', transport_cost_dict = None, walk_transport_cost = 4.5, factor = 1, default = None):
    """
    Function for adding a transport_cost value to edge dictionaries. Ensure any GeoDataFrames / graphs are in the same projection before using function, or pass a crs.

    For now no default transport costs -- if you don't provide a dictionary, this function will throw an error.

    :param G: a graph containing one or more nodes
    :param distance_tag: the key in the dictionary for the field currently
               containing a distance in meters
    :param road_col: key for the road type in the edge data dictionary
    :param graph_type: set to either 'drive' or 'walk'. IF walk - will set transport_cost = walking transport_cost across all segment, using the supplied walk_transport_cost. IF drive - will use a transport_cost dictionary for each road type, or defaults as per the note below.
    :param transport_cost_dict: transport_cost dictionary to use. Currency neutral, assumes you use local currency. If not supplied, throws error and fails.
    :param walk_transport_cost: specify a walk_transport_cost in $ / kg / km. Default = ($5.90 USD / day) / 10 hours in a working day) / (30kg) / (4 km / h)
    :param factor: allows you to scale up / down distances if saved in a unit other than meters. Set to 1000 if length in km.
    :param default: if highway type not in the transport_cost_dict, use this road class as an in-fill value for transport_cost. Assumes 'residential' if none provided
    :returns: The original graph with a new data property for the edges called 'transport_cost'
    """

    if type(G) == nx.classes.multidigraph.MultiDiGraph or type(G) == nx.classes.digraph.DiGraph:
        pass
    else:
        raise ValueError('Expecting a graph or geodataframe for G!')

    import warnings

    try:
        # checks the first edge to see if the 'transport_cost' attribute already exists
        if list(G.edges(data = True))[0][2]['transport_cost']:
          warnings.warn('Are you sure you want to convert length to transport_cost? This graph already has a transport_cost attribute')
    except:
        pass

    G_adj = G.copy()

    for u, v, data in G_adj.edges(data=True):

        orig_len = data[distance_tag] * factor

        # Note that this is a MultiDiGraph so there could
        # be multiple indices here, I naively assume this is not
        # the case
        data['length'] = orig_len

        # get appropriate transport_cost limit
        if graph_type == 'walk':
            transport_cost = walk_transport_cost

        elif graph_type == 'drive':

            if transport_cost_dict == None:
                raise ValueError('Expecting a transport cost dictionary displaying the Vehicle Operating Cost per KG')

            highwayclass = data[road_col]

            if type(highwayclass) == list:
                highwayclass = highwayclass[0]

            if highwayclass in transport_cost_dict.keys():
                transport_cost = transport_cost_dict[highwayclass]
            else:
                if default == None:
                    transport_cost = transport_cost_dict['residential']
                else:
                    transport_cost = transport_cost_dict[default]

        else:
            raise ValueError('Expecting either a graph_type of "walk" or "drive"!')

        # multiply generic road value transport cost by the segment length to get final transport cost
        TC_for_segment_length = (orig_len / 1000) * transport_cost
        data['transport_cost'] = TC_for_segment_length

        # And state the mode, too
        data['mode'] = 'transport cost'

    return G_adj


In [28]:
# gn.example_edge(largest_G,n=5)

In [29]:
# Extracted ($ / kg) / km values from the default RED maodel with "Fair" road conditions (IRI 6 -- to account for poor roads quality)
# These are only for flat terrain
# Light truck = 2 MT, Medium truck = 7 MT, Heavy Trucky = 16 MT

BDT = 84.81 # as of March 12,2021. Use to convert RED $ amounts to BDT

light_trucks = 0.0001115915 / BDT
medium_trucks = 0.0000479326 / BDT
heavy_trucks = 0.0000309086 / BDT

# arbitrary values for path and ferry edge cases must be replaced

moto_transport = (light_trucks * 2000) / 80
ferry_trucks = medium_trucks * 5 # arbitrary modifier to account for time lost waiting for ferries and their slow speed

In [30]:
# Assign the values of the type of vehicle most likely to use a given route

TC_kg_per_km_mod_dct = {'ferry':ferry_trucks,
                        'pier':ferry_trucks,
                        'path':moto_transport,
                        'track':moto_transport,
                        'service':light_trucks,
                        'living_street':light_trucks,
                        'road':light_trucks,
                        'residential':light_trucks,
                        'unclassified':light_trucks,
                        'tertiary':medium_trucks,
                        'secondary':medium_trucks,
                        'primary':medium_trucks,
                        'trunk':medium_trucks,
                        'motorway':medium_trucks,
                        'tertiary_link':medium_trucks,
                        'secondary_link':medium_trucks,
                        'primary_link':medium_trucks,
                        'trunk_link':medium_trucks,
                        'motorway_link':medium_trucks}

Now run the defined function against the defined dict

In [31]:
G_clean_TC = convert_network_to_transport_cost(largest_G, # G_salted or G_clean
                                      distance_tag = 'length',
                                      graph_type = 'drive',
                                      road_col = 'infra_type',
                                      transport_cost_dict = TC_kg_per_km_mod_dct ,
                                      factor = 1000
                                     )

Finally, before saving down, we reset all node IDs to integers to aid the graphtool step

In [32]:
G_clean_TC = nx.convert_node_labels_to_integers(G_clean_TC) # G_clean vs G_salted

Check outputs

In [33]:
gn.example_edge(G_clean_TC,n=10)

(0, 24347, {'Wkt': 'LINESTRING (89.22178599999999 22.9686986, 89.2216605 22.9687017, 89.2211562 22.9686029, 89.22057150000001 22.9684942)', 'id': 350244, 'infra_type': 'unclassified', 'osm_id': '556994112', 'key': 'edge_350244', 'length': 126.88824810020613, 'Type': 'legitimate', 'unique_id': 0, 'transport_cost': 3.339147808580723e-07, 'mode': 'transport cost'})
(0, 129963, {'Wkt': 'LINESTRING (89.22057150000001 22.9684942, 89.2206949 22.9676299, 89.22066270000001 22.9672644, 89.2205232 22.9669334, 89.22041590000001 22.9667359, 89.22050179999999 22.9663407, 89.22059299999999 22.9659999, 89.22059299999999 22.965916, 89.22043739999999 22.9659258, 89.22001899999999 22.9658764, 89.2196757 22.9658369, 89.2196757 22.965669, 89.2198473 22.9652146, 89.2203033 22.9645725, 89.220432 22.964202, 89.22043739999999 22.9637723, 89.22032369999999 22.9631695)', 'id': 350243, 'infra_type': 'unclassified', 'osm_id': '556994109', 'key': 'edge_350243', 'length': 704.0002354559285, 'Type': 'legitimate', 'un

Save down

In [34]:
gn.save(G_clean_TC, 'final_current_G_TC_{}m_{}'.format(simplif_meters,prod_date), 'final', nodes = True,  edges = True)